## 1. Agent的构建思路以及流式输出

In [113]:
# pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple
# pip install pypdfium2
# pip install Pillow
# pip install dashscope
# pip install rapidocr-onnxruntime
# pip install pypdf -i https://pypi.tuna.tsinghua.edu.cn/simple
# pip install pydantic fastapi -i https://pypi.tuna.tsinghua.edu.cn/simple
# pip install aiosqlite -i https://pypi.tuna.tsinghua.edu.cn/simple
# pip install -U langchain langchain-openai langchain-community "langgraph-checkpoint-sqlite>=1.1.0" -i https://pypi.tuna.tsinghua.edu.cn/simple


In [114]:
import os 
import random 
import uuid
from pathlib import Path 
import asyncio
import aiosqlite

def patch_aiosqlite_is_alive():
    """补丁：为aiosqlite.Connection添加is_alive方法"""
    if not hasattr(aiosqlite.Connection, 'is_alive'):
        def is_alive(self):
            """Return True if the underlying sqlite3 connection exists."""
            return self._connection is not None
        aiosqlite.Connection.is_alive = is_alive

patch_aiosqlite_is_alive()

from langchain_openai.chat_models import ChatOpenAI # 大脑
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver # 记忆
from langchain.tools import tool # 工具
from langchain.agents import create_agent
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, SystemMessage
from langgraph.types import StateSnapshot, PregelTask

In [115]:
# 1. 初始化大脑
model = ChatOpenAI(
    model="deepseek-chat",
    base_url="https://api.deepseek.com", 
    api_key="sk-14a5fe04297040c28b4f91c5dfa938a1", 
)

# 2. 初始化记忆与记忆管理
user_id = "user_001" # 用户id
session_id = str(uuid.uuid4()) # 单轮会话id
memory_dir = Path("./memories") # 存放所用用户会话文件的根目录
user_memory_dir = memory_dir / user_id
user_memory_dir.mkdir(parents=True, exist_ok=True)
memory_uri = user_memory_dir / f"session_{session_id}.db"#  由 user_id 和 session_id 决定

# 3. 初始化工具
@tool
def get_weather(location: str) -> str:
    """
    Get weather information for a given location.
    Args:
        location: The city to query weather for (e.g. "Beijing")
    Returns:
        A string of mock weather information
    """
    weather = random.choice(["Sunny", "Cloudy", "Rainy"])
    temp = random.randint(5, 30)
    return f"Weather in {location}: {weather}, Temperature: {temp}°C"

tools = [get_weather]

# 4. 系统提示词
system_prompt = "你是一个乐于助人的智能助手"

In [116]:
from langchain.tools import BaseTool
from langchain.chat_models import BaseChatModel 
from pydantic import BaseModel, Field 
from typing import List, Union
from pathlib import Path

class AgentConfig(BaseModel):  
    model: BaseChatModel = Field(..., description="Such as ChatOpenAI")
    tools: List[BaseTool] = Field(default_factory=list, description="List of callable langchain tools")
    memory_uri: Union[str, Path] = Field(..., description="The Path to SQLite Database to memory storage")
    system_prompt: str = Field(..., description="system_prompt")
    
    class Config:
        arbitrary_types_allowed = True
    
    @property
    def memory_uri_str(self) -> str:
        return str(self.memory_uri)

C:\Users\34943\AppData\Local\Temp\ipykernel_21476\1468383473.py:7: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class AgentConfig(BaseModel):


In [117]:
from typing import List, Union, Set, Dict, Any, AsyncGenerator

class AgentService:
    def __init__(self, agent_config: AgentConfig):
        self.agent_config = agent_config  
    
    async def call_agent(
        self,
        query: str,  # user query
        session_id: str,  # 1 session-id <-> 1 memory
    ) -> AsyncGenerator[Dict[str, Any], None]:

        tool_call_ids: Set[str] = set()  # 统计调用的工具id
        total_input_tokens: int = 0  # total_input_tokens (including function call message)
        total_output_tokens: int = 0  # total_output_tokens
        usage: Dict[str, int] | None = None  # token usage
        
        # extract agent's config from `AgentConfig`
        model = self.agent_config.model
        tools = self.agent_config.tools
        memory_uri = self.agent_config.memory_uri_str  
        system_prompt = self.agent_config.system_prompt
        
        async with AsyncSqliteSaver.from_conn_string(memory_uri) as checkpointer:
            await checkpointer.setup()
            agent = create_agent(
                model=model, 
                tools=tools, 
                checkpointer=checkpointer, 
                system_prompt=system_prompt
            )
            config = {
                "configurable": {
                    "thread_id": session_id
                }
            }
            
            async for chunk in agent.astream({
                "messages": [
                    {
                        "role": "user", 
                        "content": query
                    }
                ]
            }, config=config, stream_mode="messages"):
                message = chunk[0]
                state = await agent.aget_state(config=config)
                
                # starting token analyzation
                tasks = state.tasks
                for task in tasks:
                    if task.result is not None: 
                        for msg in task.result["messages"]:
                            if isinstance(msg, AIMessage) and hasattr(msg, 'usage_metadata'):
                                usage_metadata = msg.usage_metadata
                                current_input = usage_metadata.get("input_tokens", 0)
                                current_output = usage_metadata.get("output_tokens", 0)
                                if current_input not in [0, total_input_tokens]:
                                    total_input_tokens += current_input
                                if current_output not in [0, total_output_tokens]:
                                    total_output_tokens += current_output
                
                # function call message
                if isinstance(message, AIMessage) and message.tool_calls:
                    for tool_call in message.tool_calls: 
                        tool_call_id = tool_call.get("id")
                        tool_name = tool_call.get("name")
                        if tool_name and tool_call_id and tool_call_id not in tool_call_ids:
                            tool_info = f"{tool_name}"
                            yield {
                                "type": "tool_call", 
                                "content": {
                                    "tool_name": tool_name, 
                                    "tool_call_id": tool_call_id, 
                                }
                            }
                            tool_call_ids.add(tool_call_id)
            
                # completion message
                elif isinstance(message, AIMessage) and message.content: 
                    for char in message.content: 
                        yield {
                            "type": "content", 
                            "content": char
                        }
                        
            
                # tool response message
                elif isinstance(message, ToolMessage):
                    called_tool_name = message.name 
                    called_tool_content = message.content
                    yield {
                        "type": "tool_result", 
                        "content": {
                            "called_tool_name": called_tool_name, 
                            "called_tool_content": called_tool_content
                        }
                    }
            
            # yield token statisitics
            total_tokens = total_input_tokens + total_output_tokens
            usage = {
                "input_tokens": total_input_tokens, 
                "output_tokens": total_output_tokens, 
                "total_tokens": total_tokens
            }
            yield {
                "type": "usage",
                "content": usage
            }

In [118]:
# 1：初始化用户和会话信息
user_id = "user_001"
session_id = str(uuid.uuid4())

# 2. 初始化agent
agent_config = AgentConfig(
	model=model, 
	tools=tools, 
	memory_uri=memory_uri, 
	system_prompt=system_prompt
)
agent_service = AgentService(agent_config=agent_config)
# 3. 流式输出响应
query="你好, 请帮我看看深圳和杭州的天气。"
generator = agent_service.call_agent(
	query=query,
	session_id=session_id
)
async for chunk in generator:
	if chunk["type"] == "content":
		print(chunk["content"], end="", flush=True)
	elif chunk["type"] == "tool_call":
		print(chunk["content"])
	elif chunk["type"] == "tool_result":
		print(chunk["content"])
	elif chunk["type"] == "usage":
		print(f"输入Token: {chunk['content']['input_tokens']}")
		print(f"输出Token: {chunk['content']['output_tokens']}")
		print(f"总Token: {chunk['content']['total_tokens']}")

我来帮您查看深圳和杭州的天气情况。{'tool_name': 'get_weather', 'tool_call_id': 'call_00_mXEtdZZImPAX86nOSwvNv2ka'}
{'called_tool_name': 'get_weather', 'called_tool_content': 'Weather in 深圳: Sunny, Temperature: 13°C'}
{'tool_name': 'get_weather', 'tool_call_id': 'call_00_RX8JYVQqAe4sokbZ4YtVIG8e'}
{'called_tool_name': 'get_weather', 'called_tool_content': 'Weather in 杭州: Sunny, Temperature: 28°C'}
根据查询结果：

**深圳天气**：晴天，温度13°C

**杭州天气**：晴天，温度28°C

两个城市今天都是晴天，但温度差异较大。深圳相对凉爽（13°C），而杭州比较温暖（28°C）。如果您需要出行，建议根据目的地温度准备合适的衣物。输入Token: 1283
输出Token: 166
总Token: 1449


## 2. 基于RAG的多模态PDF解析

In [2]:
from langchain_core.documents.base import Blob
from langchain_community.document_loaders.parsers.pdf import PyPDFium2Parser
from langchain_community.vectorstores import FAISS  # pip install faiss-cpu
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_core.documents import Document
from typing import List, Dict, Any, Optional, Callable
import uuid  
import tiktoken  # pip install tiktoken 
import pickle
from pathlib import Path
import shutil


class VectorDataBase(FAISS): 

    def __init__(
        self, 
        embedding_function, 
        index, 
        docstore, 
        index_to_docstore_id, 
        relevance_score_fn: Optional[Callable[[float], float]] = None, 
        normalize_L2: bool = False, 
        distance_strategy: DistanceStrategy = DistanceStrategy.EUCLIDEAN_DISTANCE, 
        id: Optional[str] = None  
    ):
        # 调用父类FAISS的初始化（严格匹配参数顺序和名称）
        super().__init__(
            embedding_function=embedding_function,
            index=index,
            docstore=docstore,
            index_to_docstore_id=index_to_docstore_id,
            relevance_score_fn=relevance_score_fn,
            normalize_L2=normalize_L2,
            distance_strategy=distance_strategy
        )

        # 初始化自定义id（不传则自动生成UUID）
        self.id = id or str(uuid.uuid4())

    def save_local(self, folder_path: str, index_name: str = "index") -> None:
        super().save_local(folder_path, index_name)
        path = Path(folder_path)
        path.mkdir(exist_ok=True, parents=True)
        with open(path / f"{index_name}_id.pkl", "wb") as f: 
            pickle.dump({"id": self.id}, f)

    @classmethod
    def load_local(
        cls, 
        folder_path: str, 
        embeddings, 
        index_name: str = "index", 
        *, 
        allow_dangerous_deserialization: bool = False, 
        **kwargs
    ) -> "VectorDataBase":
        
        # 1. 加载原生FAISS实例
        base_instance = super().load_local(
            folder_path=folder_path,
            embeddings=embeddings,
            index_name=index_name,
            allow_dangerous_deserialization=allow_dangerous_deserialization,** kwargs
        )
        
        # 2. 加载自定义id
        path = Path(folder_path)
        id = None
        try:
            with open(path / f"{index_name}_id.pkl", "rb") as f: 
                id_data = pickle.load(f) 
                id = id_data.get("id")
        except FileNotFoundError:
            id = str(uuid.uuid4())
        
        # 3. 转换为VectorDataBase实例（复用原生属性，添加自定义id）
        instance = cls(
            embedding_function=base_instance.embedding_function,
            index=base_instance.index,
            docstore=base_instance.docstore,
            index_to_docstore_id=base_instance.index_to_docstore_id,
            relevance_score_fn=base_instance.override_relevance_score_fn,
            normalize_L2=base_instance._normalize_L2,
            distance_strategy=base_instance.distance_strategy,
            id=id  # 注入自定义id
        )
        return instance

    @classmethod
    def from_documents(
        cls,
        documents: list[Document],
        embedding,
        id: Optional[str] = None,  # 新增：自定义id参数（你的变量名）
        **kwargs: Any,
    ) -> "VectorDataBase":
        
        # 1. 调用父类的from_documents创建原生FAISS实例
        base_faiss = super().from_documents(documents, embedding, **kwargs)
        
        # 2. 转换为VectorDataBase实例并注入自定义id
        vector_db = cls(
            embedding_function=base_faiss.embedding_function,
            index=base_faiss.index,
            docstore=base_faiss.docstore,
            index_to_docstore_id=base_faiss.index_to_docstore_id,
            relevance_score_fn=base_faiss.override_relevance_score_fn,
            normalize_L2=base_faiss._normalize_L2,
            distance_strategy=base_faiss.distance_strategy,
            id=id  
        )
        return vector_db

class RAGTool:
    
    def __init__(
        self,
        embedding_api_key: str,
        embedding_model: str = "text-embedding-v3",
        chunk_size: int = 500,
        chunk_overlap: int = 200,
        extract_image: bool = False
    ):
        self.embedding_api_key = embedding_api_key
        self.embedding_model = embedding_model
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.extract_image = extract_image
        
        self.embedding = DashScopeEmbeddings(
            dashscope_api_key=self.embedding_api_key, 
            model=self.embedding_model
        )
        
        self.pdf_chunks: List[Dict] = []
        self.vector_db: Optional[VectorDataBase] = None  
        self.file_info: Dict[str, Any] = {}
        self.token_encoder = tiktoken.get_encoding("cl100k_base")
        self.vectorbase_usage = 0
        self.query_usage = 0
    
    def count_tokens(self, text: str) -> int:
        return len(self.token_encoder.encode(text)) if text else 0
    
    def process_pdf(self, file_content: bytes, filename: str) -> List[Dict]:
        try:
            # 1. 构建Blob
            blob = Blob.from_data(
                data=file_content,
                path=filename
            )
            
            # 2. PDF解析
            parser = PyPDFium2Parser(
                extract_images=self.extract_image,
                images_inner_format="html-img"
            )
            documents = list(parser.parse(blob))
            
            # 3. 拼接完整文本
            full_content = "\n\n".join([doc.page_content for doc in documents])
            
            # 4. 文本分块
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.chunk_size,
                chunk_overlap=self.chunk_overlap,
                separators=["\n\n", "\n", " ", ""]
            )
            all_chunks = splitter.split_text(full_content)
            
            # 5. 格式化分块结果
            chunks = []
            self.vectorbase_usage = 0
            total_chunks = len(all_chunks)
            for i, chunk_text in enumerate(all_chunks):
                clean_text = chunk_text.strip()
                if not clean_text:
                    continue
                chunk_token = self.count_tokens(clean_text)
                self.vectorbase_usage += chunk_token
                chunks.append({
                    "id": str(uuid.uuid4()),
                    "content": clean_text,
                    "metadata": {
                        "filename": filename,
                        "chunk_index": i,
                        "total_chunks": total_chunks,
                        "token": chunk_token
                    }
                })
            
            self.pdf_chunks = chunks
            self.file_info = {
                "filename": filename,
                "chunk_count": len(chunks),
                "file_size": len(file_content),
                "extract_image": self.extract_image,
                "vectorbase_usage": self.vectorbase_usage
            }
            
            return chunks
        
        except Exception as e:
            raise RuntimeError(f"Failed to chunk PDF: {str(e)}")
    
    def build_vector_db(self, vector_db_id: Optional[str] = None) -> VectorDataBase:
        if not self.pdf_chunks:
            raise RuntimeError("Please process PDF with process_pdf first before building vector database")
        
        try:
            langchain_docs = []
            for chunk in self.pdf_chunks:
                langchain_doc = Document(
                    page_content=chunk["content"],
                    metadata={
                        "id": chunk["id"],
                        "filename": chunk["metadata"]["filename"],
                        "chunk_index": chunk["metadata"]["chunk_index"],
                        "total_chunks": chunk["metadata"]["total_chunks"]
                    }
                )
                langchain_docs.append(langchain_doc)
            
            self.vector_db = VectorDataBase.from_documents(
                documents=langchain_docs,
                embedding=self.embedding,
                id=vector_db_id
            )
            self.file_info["vector_db_id"] = self.vector_db.id # 新增一个键值对
            
            return self.vector_db
        
        except Exception as e:
            raise RuntimeError(f"Failed to build FAISS vector database: {str(e)}")
    
    def semantic_search(self, query: str, top_k: int = 3) -> List[Dict]:
        if not self.vector_db:
            raise RuntimeError("Please build vector database first before performing search")
        
        try:
            query_token = self.count_tokens(query)
            self.query_usage += query_token
            
            results_with_score = self.vector_db.similarity_search_with_score(query, k=top_k)
            
            formatted_results = []
            for doc, score in results_with_score:
                formatted_results.append({
                    "id": doc.metadata["id"],
                    "content": doc.page_content,
                    "metadata": doc.metadata,
                    "similarity_score": round(score, 4),
                    "vector_db_id": self.vector_db.id  # 返回你的自定义id
                })
            
            return formatted_results
        
        except Exception as e:
            raise RuntimeError(f"Semantic search failed: {str(e)}")
    
    def save_vector_db(self, folder_path: str, index_name: str = "index") -> None:
        if not self.vector_db:
            raise RuntimeError("Please build vector database first before saving")
        self.vector_db.save_local(folder_path, index_name)
        print(f"Vector database (id: {self.vector_db.id}) saved to: {folder_path}")
    
    def load_vector_db(self, id: str) -> VectorDataBase:
        try:
            folder_path = f"./{id}"
            index_name = id 
            self.vector_db = VectorDataBase.load_local(
                folder_path=folder_path,
                index_name=index_name,
                embeddings=self.embedding, 
                allow_dangerous_deserialization=True # 生产环境需谨慎
            )
            if self.vector_db.id != id:
                raise RuntimeError(f"Unmatched: {self.vector_db.id} != {id}")
            self.file_info["vector_db_id"] = id 
            return self.vector_db
        except FileNotFoundError: 
            raise RuntimeError(f"VectorBase-{id} not found in {folder_path}")
        except Exception as e: 
            raise RuntimeError(f"Failed")
    
    def get_token_stats(self) -> Dict:
        return {
            "vectorbase_usage": self.vectorbase_usage,
            "query_usage": self.query_usage,
            "total_usage": self.vectorbase_usage + self.query_usage
        }
    
    def get_file_info(self) -> Dict:
        return self.file_info
    
    def clear(self):
        self.pdf_chunks = []
        self.vector_db = None
        self.file_info = {}
        self.vectorbase_usage = 0
        self.query_usage = 0

    def clear_disk(self, by_id: bool = False, vector_db_id: Optional[str] = None) -> None:
        try:
            # by_id=True：按ID删除（传ID删指定，不传ID报错）
            if by_id:
                if not vector_db_id:
                    raise RuntimeError("vector_db_id is required when by_id=True")
                target_path = Path(f"./{vector_db_id}")
                if not target_path.exists():
                    raise RuntimeError(f"Vector database {vector_db_id} not found on disk")
                if target_path.is_dir():
                    shutil.rmtree(target_path)
                    print(f"Successfully deleted vector database {vector_db_id} from {target_path}")
                elif target_path.is_file():
                    target_path.unlink()
                    print(f"Successfully deleted vector database file {vector_db_id}")
            # by_id=False：强制批量删除（忽略vector_db_id，直接删所有）
            else:
                deleted_count = 0
                current_dir = Path(".")
                for item in current_dir.iterdir():
                    if item.is_dir():
                        try:
                            uuid.UUID(item.name)
                            shutil.rmtree(item)
                            deleted_count += 1
                            print(f"Deleted vector database: {item}")
                        except ValueError:
                            continue
                if deleted_count == 0:
                    print("No vector databases found on disk to delete")
                else:
                    print(f"Successfully deleted {deleted_count} vector databases")
        except PermissionError:
            raise RuntimeError("Permission denied: cannot delete vector database files")
        except Exception as e:
            raise RuntimeError(f"Failed to clear vector database from disk: {str(e)}")
        
"""
参数组合	执行行为
by_id=False	忽略 vector_db_id，批量删除所有 UUID 格式向量库文件夹
by_id=False, vector_db_id="xxx"	仍批量删除所有，传入的 ID 被忽略
by_id=True	必须传 vector_db_id，否则报错
by_id=True, vector_db_id="xxx"	仅删除指定 ID 的向量库
"""

'\n参数组合\t执行行为\nby_id=False\t忽略 vector_db_id，批量删除所有 UUID 格式向量库文件夹\nby_id=False, vector_db_id="xxx"\t仍批量删除所有，传入的 ID 被忽略\nby_id=True\t必须传 vector_db_id，否则报错\nby_id=True, vector_db_id="xxx"\t仅删除指定 ID 的向量库\n'

In [ ]:
# 1. 初始化工具类
rag_tool = RAGTool(
    embedding_model="text-embedding-v3",
    embedding_api_key="your-api-key",
    chunk_size=500,
    chunk_overlap=200,
    extract_image=False
)

# 2. 读取PDF文件并分块
f= open("./test2.pdf", "rb")
file_content = f.read() 
f.close()
chunks = rag_tool.process_pdf(
    file_content=file_content, 
    filename="test.pdf"
)
print(f"PDF分块完成, 共{len(chunks)}块")

# 3. 构建FAISS向量库并检索知识
vector_db = rag_tool.build_vector_db()
knowledge_base_id = vector_db.id
rag_tool.save_vector_db(folder_path=f"./{knowledge_base_id}", index_name=f"{knowledge_base_id}") # 根据id查询必须: 保存向量数据库到本地
rag_tool.load_vector_db(id=knowledge_base_id) # 先加载
results = rag_tool.semantic_search(query="如何测量细丝直径？", top_k=5) # 再查询
print(results, type(results))
for i, res in enumerate(results, 1):
    print(f"【结果{i}】")
    print(f"相似度：{res['similarity_score']}")
    print(f"内容：{res['content'][:10]}...")
    print(f"所属文件：{res['metadata']['filename']}")
print("Token消耗情况: ", rag_tool.get_token_stats())
rag_tool.clear()
# rag_tool.clear_disk()

### 步骤 1：更新系统并安装 Redis
&emsp;&emsp;首先在 WSL 终端执行以下命令：
- 1. 更新系统包列表
```bash
sudo apt update && sudo apt upgrade -y
```

- 2. 安装 Redis 服务器
```bash
sudo apt install redis-server -y
``` 

- 3. 验证安装（查看版本）
```bash
redis-server --version
```
安装成功后会输出类似：Redis server v=6.0.16 sha=00000000:0 malloc=jemalloc-5.2.1 bits=64 build=abc1234

### 步骤 2：配置 Redis（关键）
&emsp;&emsp;默认配置可能限制远程访问 / 持久化，需要修改配置文件：
- 2.1 编辑 Redis 配置文件
```bash
sudo nano /etc/redis/redis.conf
```
- 2.2 修改核心配置项（按需求调整）
&emsp;&emsp;在配置文件中找到以下项，修改为对应值（用 Ctrl+W 搜索关键词）：

| 配置项          | 默认值          | 修改后            | 说明                                                 |
|-----------------|-----------------|-------------------|------------------------------------------------------|
| bind            | 127.0.0.1 ::1   | 0.0.0.0           | 允许所有 IP 访问（开发环境），生产环境需指定具体 IP   |
| protected-mode  | yes             | no                | 关闭保护模式（允许外部连接）                         |
| requirepass     | 无              | your_redis_password | （可选）设置 Redis 密码，对应代码中的 password      |
| daemonize       | yes             | yes               | 后台运行（保持默认即可）                             |
| save            | 900 1           | 保留默认          | 持久化策略（自动保存数据到磁盘）                     |

示例修改：
```ini
# 允许所有IP访问
bind 0.0.0.0

# 关闭保护模式
protected-mode no

# （可选）设置密码（替换为你的密码）
requirepass my_redis_123456
```

- 2.3 保存并退出
按 Ctrl+O 保存 → 按 Enter 确认 → 按 Ctrl+X 退出 nano 编辑器。
步骤 3：启动 / 重启 Redis 服务
bash
运行
重启 Redis 服务（使配置生效）
```bash
sudo systemctl restart redis-server
```

# 设置开机自启（可选）
```bash
sudo systemctl enable redis-server
```
# 查看 Redis 状态
```bash
sudo systemctl status redis-server
```
✅ 正常状态会显示：active (running)



In [ ]:
# test redis 
import redis

# ===================== 配置项（替换为你的 WSL Redis 信息） =====================
REDIS_HOST = "localhost"  # WSL IP（如 172.17.0.2），本地测试填 localhost 即可
REDIS_PORT = 6379         # Redis 默认端口
REDIS_PASSWORD = "your-redis-password"       # 你设置的 Redis 密码（无则留空）
REDIS_DB = 0              # 使用的数据库编号

def test_redis_connection():
    """Test Redis connection and basic operations"""
    try:
        # 1. 初始化 Redis 客户端（与 RAG API 配置一致）
        redis_client = redis.Redis(
            host=REDIS_HOST,
            port=REDIS_PORT,
            db=REDIS_DB,
            password=REDIS_PASSWORD,
            decode_responses=True,  # 与 API 保持一致的配置
            socket_timeout=5        # 超时时间，避免卡死
        )
        
        # 2. 测试连接（核心验证）
        pong = redis_client.ping()
        if pong:
            print("✅ Redis connection successful!")
        else:
            print("❌ Redis ping failed!")
            return False
        
        # 3. 测试基础操作（模拟 API 中的数据读写）
        # 写入测试数据
        test_key = "rag:test:connection"
        redis_client.set(test_key, "redis_connection_ok", ex=300)  # 5分钟过期
        
        # 读取测试数据
        test_value = redis_client.get(test_key)
        print(f"✅ Test data read: {test_key} = {test_value}")
        
        # 删除测试数据
        redis_client.delete(test_key)
        print(f"✅ Test data deleted: {test_key}")
        
        # 4. 输出 Redis 服务器信息（可选）
        redis_info = redis_client.info("server")
        print(f"\n📌 Redis Server Info:")
        print(f"   Version: {redis_info.get('redis_version')}")
        print(f"   Mode: {redis_info.get('redis_mode')}")
        print(f"   Uptime: {redis_info.get('uptime_in_seconds')} seconds")
        
        return True
    
    except redis.exceptions.ConnectionError:
        print("❌ Connection Error: Could not connect to Redis server")
        print("   Check if Redis is running on WSL, and host/port is correct")
        return False
    except redis.exceptions.AuthenticationError:
        print("❌ Authentication Error: Wrong Redis password")
        return False
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")
        return False

if __name__ == "__main__":
    print("===== Testing Redis Connection =====")
    test_redis_connection()

===== Testing Redis Connection =====
✅ Redis connection successful!
✅ Test data read: rag:test:connection = redis_connection_ok
✅ Test data deleted: rag:test:connection

📌 Redis Server Info:
   Version: 7.0.15
   Mode: standalone
   Uptime: 24 seconds
